In [1]:
def mod_mul_standard(a, b, p):
    result = (a * b) % p
    if result > (p>>1):
        result -= p
    return result

def mod_add(a, b, p):
    result = (a + b) 
    if result > (p>>1):
        result -= p
    if result < -(p>>1):
        result += p
    return result

def mod_sub(a, b, p):
    result = (a - b) 
    if result > (p>>1):
        result -= p
    if result < -(p>>1):
        result += p
    return result

def check_arrays(a,b,p):
    if a == b:
        print("exactly equal")
        return True
    print("not exactly equal")

    if len(a) != len(b):
        print("lengths not equal")
        return False


    for i in range(len(a)):
        if (a[i] - b[i]) % p != 0:
            print("element", i, "not equal even in modulus class")
            return False
    print("But equal in modulus class")
    return True

In [2]:
# LinearConv.
def Linear_Conv(f,g,p):
    if p != 0:
        
        n = len(f)
        result = [0]*(2*n-1)
        for i in range(n):
            for j in range(n):
                result[i+j] += (f[i]*g[j] % p)
        return result

    else:

        n = len(f)
        result = [0]*(2*n-1)
        for i in range(n):
            for j in range(n):
                result[i+j] += (f[i]*g[j])
        return result



def NegaCyclic_Conv(f,g,p):
    if p != 0:

        n = len(f)
        result = Linear_Conv(f,g,p)
        for i in range(n-1):
            result[i] -= result[i+n] % p
        return result[:n]

    else:

        n = len(f)
        result = Linear_Conv(f,g,p)
        for i in range(n-1):
            result[i] -= result[i+n] 
        return result[:n]

In [3]:
def Mu(b,q):
    mu = 2*int(((b << 31)/(2*q))+0.5)
    return mu

def Barrett_mul(a, b, p, mu_b):
    # Perform the multiplication
    z = (a * b) % (1 << 32)
    if z >= (1 << 31):
        z -= (1 << 32)  # Convert to signed 32-bit integer

    # Perform the multiplication for mu_b and get the high 32 bits
    temp = (a * (mu_b << 1)) % (1 << 64)
    t_high = (temp >> 32) % (1 << 32)
    if t_high >= (1 << 31):
        t_high -= (1 << 32)  # Convert to signed 32-bit integer
    
    result = z - t_high * p
    # Simulate 32-bit signed integer overflow for the result
    result %= (1 << 32)
    if result >= (1 << 31):
        result -= (1 << 32)  # Convert to signed 32-bit integer

    return result

def Barrett_red(A, N, V, i=25):
    # Step 1: Doubling multiplication, high half (sqdmulh equivalent)
    T0 = (A * V) >> (i + 1)
    
    # Step 2: Signed shift right with rounding (srshr equivalent)
    T1 = (T0 + (1 << (i - 2))) >> (i - 1)
    
    # Step 3: Subtract modulus (mls equivalent)
    z = A - (T1 * N)
    
    # Ensure result is in range -N/2 ≤ z < N/2
    if z >= N // 2:
        z -= N
    elif z < -N // 2:
        z += N
    
    return z


def number_order(temp, p):
    maximum = max([abs(x) for x in temp])

    degree = 0
    while True:
        if degree*(p>>1) < maximum < (degree+1)*(p>>1):
            return degree
        degree += 1


In [4]:
N = 101  # Example odd modulus
i = N.bit_length()  # Bit length of N
R = 2**31  # Radix for 32-bit signed numbers
V = (2**(i - 2) * R) // N  # Multiplier as described

A = 123456  # Example number to reduce

# Perform Barrett Reduction
z = Barrett_red(A, N, V, i)
print(f"Result of Barrett Reduction: z = {z}")

Result of Barrett Reduction: z = -517811870469


In [5]:
def CT_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    # Second layer
    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    return t0, t1, t2, t3

def GS_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    # Second layer
    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    return t0, t1, t2, t3

In [6]:
def forward_10(temp,ptr,P,omegas,mu_omegas):
    n = 1 << 10
    ptrf = ptr
    ptrg = ptrf + n


    for depth in range(0, 10, 2):
        logn_top = 10 - depth
        n = 1 << logn_top
        hhn = n >> 2
        power_index = 1 << depth

        for j in range(0, 1 << depth):
            for i in range(0, hhn):
                temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P, temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
                temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P, temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
            power_index += 1
    
def backward_10(temp,ptr,P,omegas,mu_omegas):
    n = 1 << 10
    ptrf = ptr
    ptrg = ptrf + n


    for depth in range(8, -1, -2):
        logn = 10 - depth
        n = 1 << logn
        hhn = n >> 2
        power_index = 2*(1 << depth)-1

        for j in range(0, 1 << depth):
            for i in range(0, hhn):
                temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P, temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
                temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P, temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
            power_index -= 1

    

In [7]:
from data import *

logn_top = 10
n = 1 << logn_top
temp = [0] *  (8 * n)

temp[0:n] = f[0:n]
temp[n:2*n] = g[0:n]

ptrf = 0
ptrg = n

In [8]:
forward_10(temp,ptrf,P[0],omegas,mu_omegas)


In [9]:
print(temp)

[-435656, -5400622, 14551170, -14628658, -10278786, -3879386, 11016177, 15598448, 11740741, 9070733, -14620289, -717763, -12857682, -12499730, 8954476, -8410983, 4504662, -10847729, 13728358, -6014042, -10551078, -5842633, -11905405, -15704585, 10391451, 6978326, -3955142, 253091, 14565644, 3993063, -8742101, -16478465, -4558249, 10764973, -5699010, -257190, 7613289, 278392, 2168999, -1336145, -1546345, -7864933, 7527473, -8671020, 1258819, -14560521, -10996907, 9879745, -2352077, -12542855, -12787472, -1062464, 8648262, -13193259, 8329229, 5823247, 11335916, -2941838, 8585768, -16368172, 2754043, 4583696, -8098344, 12306646, -9691065, -5836568, -6777714, 10872593, -15650728, 9635981, -3984683, -3573324, 1072972, 9501936, -8470761, -6729738, 9883717, -6963115, -9277451, 14967589, -5910749, 10458373, -8323217, 5891588, -16504879, 10637871, -4245469, 15325376, -15321262, -12046611, 13872834, 1526107, -6467106, 13136344, -6467131, 8474103, -2630112, 1903282, -167874, -15228352, -4881872, 

In [10]:
print("""
temp: |--- NTT(f) under P[0] ---|--- NTT(g) under P[0] ---|--->
""")



temp: |--- NTT(f) under P[0] ---|--- NTT(g) under P[0] ---|--->



In [11]:
hn = n >> 1
ptrNf = 2*n
ptrNg = 2*n + hn
print(ptrf, ptrg,ptrNf, ptrNg)

0 1024 2048 2560


In [12]:
for i in range(hn):
    temp[ptrNf + i] = mod_mul_standard( temp[ptrf + 2*i] , temp[ptrf + 2*i + 1],P[0])
    temp[ptrNg + i] = mod_mul_standard( temp[ptrg + 2*i] , temp[ptrg + 2*i + 1],P[0])

In [13]:
print(temp[ptrNf:ptrNf+hn])

[-4655104, 16156743, -2227192, -174659, 3161240, -5463690, -1469057, -1690740, 8670825, -4440857, 16297834, -7453023, 5956977, -2989190, 11917852, 14914922, -12979220, 14809381, 3311987, 14491542, 3308396, 3237897, -8126544, -10285886, 10026299, -13867479, -2738507, 5941392, 2618326, 9276251, 12532971, 4036181, 13166283, 9958215, -10815688, 1675514, 1315895, -12862485, 12686275, 13669617, 13674493, 5858582, -9372751, -11834995, -11333223, -7831557, -12319621, -732473, -345836, 9335259, -1613194, -11895936, 13611108, 9497779, -1864851, -7999652, 3296245, -15526544, 10079417, -3315100, -2044484, -16502292, -1673979, 14682771, -1031549, 11263915, 13088649, -16663669, -9716321, -15258472, -11472828, -2513876, 706564, 8338603, 4046584, -3174367, 2393646, -948760, -7572651, 10363561, -5509393, 4160369, 14177722, 2894467, -2231227, 12989353, 13101853, 10613958, 12971260, 14099134, -15171699, -16711465, 16770041, 16171858, 15859546, -2244633, -14118076, 9054174, 328126, -10778227, 7913579, 744

In [14]:
n = hn
hn = n >> 1
ptrf = ptrNf
ptrg = ptrNg
ptrNf = ptrg + hn
ptrNg = ptrNf + hn

print(ptrf, ptrg, ptrNf, ptrNg)


2048 2560 2816 3072


In [15]:
for i in range(hn):
    temp[ptrNf + i] = mod_mul_standard( temp[ptrf + 2*i] , temp[ptrf + 2*i + 1],P[0])
    temp[ptrNg + i] = mod_mul_standard( temp[ptrg + 2*i] , temp[ptrg + 2*i + 1],P[0])

In [16]:
print(temp[ptrNf:ptrNf+hn])


[14802130, 16520350, 13615370, -5116604, -4370440, -8852444, 3331087, -12680244, -5396347, -4935103, -8067181, -13557970, -16483136, -5700561, -14040269, -13436314, 6439402, 8813211, -4387293, -4794427, -106187, -11491745, 2841114, -9537798, 13197312, -4681046, 9509494, 1684802, 1448696, 15086765, -16634590, -8932979, 6656853, -11514819, -9704191, -11838026, 15559859, 7714188, -6817767, 9277046, 12687665, -12036028, -4911377, -5964430, 8445437, 11005881, 15231169, -6840072, 5428190, -8388758, -1774326, 14294446, -12206594, 13859514, -14104019, 987910, 10493315, -6046160, 10810131, -33579, 10662786, 1353119, -13324299, -11048389, 13702124, -11526222, 12094914, 9046493, -14524318, 4380718, 1067604, -3430585, -3838500, -5990508, 12557619, -10064716, 16216509, 6877892, -9771641, 4421507, -2574160, 1455647, -3184782, -11392850, 11276873, -10202082, 13106562, 15658289, -2235709, -2589761, -5588966, -9965610, 6831886, -15847039, 13987267, -12209598, 2964647, -9320043, 13610908, 262518, -14043

In [17]:
ptrf = ptrNf
ptrg = ptrNg

depth = 8
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1


In [18]:
print(temp[ptrNf:ptrNf+hn])
h=[6270909, 13122197, -5887595, -1500799, -22572041, 3033296, 14706030, 11476388, -31956601, -12548821, 12746775, -4498110, -49660280, -14907872, 8539326, -4201537, 6070893, 13797057, 1083789, 4926091, -18294616, -2816366, 13732605, -9409415, -13839775, 5426978, 7257252, 9193992, -9032108, 1886781, -6392626, -15119707, -26400183, 2513376, 9277944, -10541871, -7817011, 11068787, 8460970, 14395964, -10224170, -14735668, 9769815, -5123791, -5707922, -10587602, -7248351, 14795623, 9559552, -12365827, -9747468, -877728, -11463189, 309436, -10207413, 5870212, 15223707, -3728300, 8460834, 7612280, -12356783, 4393350, 3170721, -9788651, -10233028, -15374484, -2325452, -12937558, -12506581, -2917107, 12310142, 3319027, -7336105, 5991992, -1732655, 7001609, -15806070, -2876672, -1467655, 12473117, -15696145, 13929581, 14873460, -9423447, -3710695, -4790912, -7405307, -16634910, -20380046, 5029619, -5114155, -5810204, -7237484, -6702490, -15286811, 5519182, 7518030, 665766, -1806123, -14344920, -24060479, 1322946, -79504, -11759279, -3084799, 2918815, 5354672, 7343147, -23159848, -3892801, 7247491, -9917017, -8566395, 14640572, 4238875, -9755820, -13251878, 5347982, 8889390, 10524119, -4482429, -4891114, -15668100, 11523922, 10979341, 1492585, -9363544, -11002937, -22970913, 2687342, 8967053, 16367681, -7450047, -6751643, 7748399, -6562425, -13123634, -4722696, 2291411, 1321900, 6761270, 9147994, -9335493, 9749436, 12912694, -7422359, -9528444, 4548594, -3172617, -13477258, 10721433, -868023, 10029012, -14449149, 15365680, 6340309, -14512375, -7064590, 16284279, -4412679, -29580460, -5573697, -5842972, -6233095, 1707688, -15828026, 858896, 7938410, -17355848, -15339423, 10666957, -2055148, -32292014, 9985461, -6982659, -9114855, -15984781, 6982136, -10355751, -15483042, -37907097, 3116299, 10346395, 16594301, -12783558, 6472079, 758073, 5124450, -8076547, 5220319, -8157470, 7216629, 6330131, -2412920, 12454214, 4403449, -2984271, -20724448, 14958279, -13982572, -12792688, -3745688, -3206702, 7777158, -7558023, -17535647, 13862491, -6344096, -36365723, 16713276, -11347487, 15490826, 10314761, 11628695, 5995464, -13515760, -18855962, 2614399, -10913643, -8812809, 10724905, 1316665, -890184, 9752720, 13498654, -4640341, -11996024, -10535754, -6456729, -4411431, -11240794, -11372045, -31449521, -2717739, -8979964, -1953911, 9548625, 1294613, -4486632, 1193408, -24247045, -28599748, 10841478, 15870185, -12158538, -8495262, 2297436, 1120704, 4499797, 8061253, 6495787, -9663799, 6522953, -16481572, -2120687, -69243] 
check_arrays(temp[ptrNf:ptrNf+hn],h,P[0])

difference = [temp[ptrNf + i] - h[i] for i in range(hn)]
print(difference)


for i in range(hn):
    # if h[i] > (P[0]>>1):
    #     h[i] -= P[0]
    if h[i] < -(P[0]>>1):
        h[i] += P[0]


print("the order of h is:", number_order(h,P[0]))
difference = [temp[ptrNf + i] - h[i] for i in range(hn)]


[6270909, 13122197, 27662742, 32049538, 10978296, 3033296, 14706030, 11476388, 1593736, 21001516, 12746775, 29052227, -16109943, -14907872, 8539326, 29348800, 6070893, 13797057, 1083789, 4926091, 15255721, -2816366, 13732605, 24140922, -13839775, 5426978, 7257252, 9193992, -9032108, 1886781, 27157711, 18430630, 7150154, 2513376, 9277944, 23008466, -7817011, 11068787, 8460970, 14395964, -10224170, -14735668, 9769815, 28426546, -5707922, -10587602, 26301986, 14795623, 9559552, -12365827, 23802869, 32672609, -11463189, 309436, 23342924, 5870212, 15223707, -3728300, 8460834, 7612280, -12356783, 4393350, 3170721, 23761686, -10233028, 18175853, 31224885, 20612779, -12506581, -2917107, 12310142, 3319027, -7336105, 5991992, 31817682, 7001609, -15806070, -2876672, 32082682, 12473117, -15696145, 13929581, 14873460, 24126890, -3710695, -4790912, 26145030, 16915427, 13170291, 5029619, 28436182, 27740133, -7237484, 26847847, 18263526, 5519182, 7518030, 665766, 31744214, 19205417, 9489858, 1322946, 

In [19]:
depth = 6
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [20]:
print(temp[ptrNf:ptrNf+hn])


[2732998, -11301200, -3445801, 1275942, 14216689, -16050215, -10818730, -14951613, 10671164, 8465250, 9561633, 27466219, 8846927, 33336067, 17834978, 2791600, -1545269, -15255887, 15681020, -10409039, 9030392, 8209621, 2524999, 9734882, 10277723, 24939078, 10716652, 24785504, 5092091, 31650832, 9308950, 21784103, -16598949, -11741107, -13289959, 13525925, -7593740, 22116945, 4455733, -7552066, 27262339, 30549574, 15477134, 30595587, 4398819, 5830069, 27361131, 32393718, 963287, -11391341, -8323326, 2816113, -31743, 6455774, 26174945, -1140932, 4183735, 23547652, 22058891, 9622681, 19298708, 18495890, 1924106, 10813356, -12331447, -15176271, 6784380, 9856195, 6700833, 14997370, -6988775, 14541001, 16313008, 23063816, 29481187, 19187239, 21133616, 24525618, 16513314, 31294835, -13474033, 7465798, -12932813, 7200958, 3181394, -8108011, 4772868, -11921147, 11015261, 18313183, 7849768, 13717717, 5618205, 9360321, 19029108, 9315356, -9236759, 1014726, 10716536, 4872268, 17570420, -6465794, 1

In [21]:
depth = 4
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [22]:
print(temp[ptrNf:ptrNf+hn])


[-14447933, -16139198, -9378066, 7208941, 15621598, -12818212, -11213390, -13909729, -14705713, -13149457, -9286364, -8181020, 4086208, -11338153, -10671509, 682103, 15942662, 17274074, -2446189, 13697775, 16444553, -11081035, 12121114, -7403883, -13343474, 29650972, 11577378, -6396512, 9363903, -725189, -4428162, 12221954, 14087764, 9959092, 18270922, 22189178, 23645370, 2750121, 7926063, 20811806, 27448500, 21198867, 18521686, 5330889, 13790698, 23303077, 334851, 22438888, 17232593, 16033078, 26612101, 28427787, 8238827, 2151022, 24376424, 26542374, 8879851, 29930670, 13272135, 9515870, 23044040, 29519191, 30167754, 20083366, 16737074, 9894278, -7335276, -10331632, 4939129, 978821, -8347301, 3568248, -14312200, -7235117, 10427247, 13422387, 13855173, 9275874, -10268343, -15409515, 13496082, -4536448, -2549890, -16474057, 3781266, -2038492, 493401, 16100669, -10688843, 10381720, 15702095, 3781154, -8503630, 9260750, 16682624, 17069677, 1048153, 18495025, 30642657, 11286478, 1808666, 2

In [23]:
depth = 2
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [24]:
print(temp[ptrNf:ptrNf+hn])
print(hn)
check_arrays(temp[ptrNf:ptrNf+hn],[-608442130, -255599374, -81835534, -127493135, -83903752, -65047304, -43918356, -85050380, -35289095, -65075978, 15305462, -42834704, 932598, -33352717, -47475476, -32583184, -13718280, -48751632, -30507537, -4487959, -6556683, -22755849, -35525902, -21364755, -19212301, -6469137, -25664787, -30467606, -23784466, -1571854, -22101780, 7178986, 7545075, -1222416, -9176340, 10847472, -17722138, 12809202, -11342352, -21405198, -29056531, -10107920, -9249548, 6374126, -1569810, -8745997, 9290483, -8703508, -33946134, -2602001, -28146704, 2699754, -14976788, 7127019, -9452557, -24275218, 16260839, -2545685, 9909481, 4007145, 8578791, 3280873, 6773230, -26318617, -1385497, -5656598, 16114668, 11210730, 8262885, -20459803, -8232725, 11873259, -17083673, 4999920, -8780309, -7686168, 12524521, -4075034, 1106668, 8037106, 4508133, 3785192, 7230188, -5910548, -7509523, -6030617, -1456658, -15135766, 3557345, 230889, -506646, 370410, -4399638, 52961, 15513063, -15896082, -13304344, 4017373, -1521686, -12924692, -9628953, -6863642, 1527019, -3231251, 11006440, 9234662, 5562349, 15015920, 3737836, -16980765, 8356839, 1505007, 2678503, -6294039, -9587989, 16366832, 407270, -3866138, -3959059, -15371544, -17052441, -15754012, 12860652, -23301136, -13602576, -7404567, 6868726, 14547949, -7695375, 14210281, 15469295, 13330160, -16706323, 4163821, 2677739, 16311020, 8469486, -10457111, 3208171, 9331439, 1369583, 1701864, -2730517, -5388824, 5892845, 1997289, 2700522, -11563283, 13210095, 5769706, -8267542, 214250, -7467534, 4813807, -8479768, 1651951, 5068785, -12279056, -7635470, -11849747, 6079731, -676625, 3194352, 5026537, 5878777, 4472301, -3003158, 8474608, -5007374, -11144463, -9026320, -8845841, -11519505, -1043468, -8154130, -12157199, -10596106, 15566332, -10804241, 5024236, 14538994, -10758415, 14286321, -11285523, 12395255, 3924208, 6486773, -10038286, -221710, -4950280, 10324471, -9057292, 8981750, 15209981, -9719051, 630773, 14290679, 11549438, 5509366, -10157062, 1307639, 10914562, -16148994, 8323832, 4862715, 3346944, -4029951, 8445952, -7328257, 5216772, -1423869, 12428030, -7500801, 5682947, 12473856, -10259972, 16072443, 10214656, -3797499, -3772937, -16276225, 16213771, 1933572, -11175934, -13301507, -2271229, 9715199, 14881535, 15876351, 10504198, -4382206, 1415685, -8503559, 2286088, -2432762, -15611641, -3530234, -11885558, 5378065, -15195895, 13654028, -1933300, 2700044, -15810039, -14399478, -839163, 13779467, -11994614, -3078908, 3070212, 9289735, -599794, -11215356, -3652853, 5177350, -9708528] ,P[0])


[-4536064, 12803322, -14734860, 6708213, 16747259, 2053370, -10368019, 15600631, -1738758, 2024696, 15305462, -9284367, 932598, 197620, -13925139, 967153, -13718280, -15201295, 3042800, -4487959, -6556683, 10794488, -1975565, 12185582, 14338036, -6469137, 7885550, 3082731, 9765871, -1571854, 11448557, 7178986, 7545075, -1222416, -9176340, 10847472, 15828199, 12809202, -11342352, 12145139, 4493806, -10107920, -9249548, 6374126, -1569810, -8745997, 9290483, -8703508, -395797, -2602001, 5403633, 2699754, -14976788, 7127019, -9452557, 9275119, 16260839, -2545685, 9909481, 4007145, 8578791, 3280873, 6773230, 7231720, -1385497, -5656598, 16114668, 11210730, 8262885, 13090534, -8232725, 11873259, 16466664, 4999920, 24770028, -7686168, 12524521, -4075034, 1106668, 8037106, 4508133, 3785192, 7230188, -5910548, -7509523, 27519720, -1456658, -15135766, 3557345, 230889, -506646, 370410, -4399638, 52961, 15513063, 17654255, -13304344, 4017373, -1521686, 20625645, -9628953, -6863642, 1527019, -32312

True

In [25]:
inv2 = 16775169
inv4 = mod_mul_standard(inv2, inv2, P[0])
mu_inv4 = Mu(inv4, P[0])

inv16 = mod_mul_standard(inv4, inv4, P[0])
mu_inv16 = Mu(inv16, P[0])
inv64 = mod_mul_standard(inv16, inv4, P[0])
mu_inv64 = Mu(inv64, P[0])
inv256 = mod_mul_standard(inv64, inv4, P[0])
mu_inv256 = Mu(inv256, P[0])
print(inv256)
print(mu_inv256)
inv1024 = mod_mul_standard(inv256, inv4, P[0])
mu_inv1024 = Mu(inv1024, P[0])




for i in range(ptrNf,ptrNf+hn):
    temp[i] = mod_mul_standard(temp[i], inv256, P[0])

print(temp[ptrNf:ptrNf+hn])


-131056
-8388606
[-17719, 836349, 1515114, 1467820, 720699, 794357, 2449564, 1240444, 779544, 1056357, 1370347, 1929573, 1314203, 1573444, 2435669, 1969618, 994861, 1906460, 2108782, 2996757, 1416004, 1090614, 1696011, 2406608, 1628680, 2202682, 2389811, 2764218, 2266100, 1828644, 2534785, 2911275, 1733201, 2092121, 2585275, 2139269, 3338229, 1884820, 2052590, 1751170, 2376562, 2057412, 1536541, 2383907, 2352876, 1669564, 1740019, 2587122, 2750630, 2217788, 1986948, 2893778, 2562617, 2780016, 1666804, 2264183, 3339919, 2742232, 3052997, 3029941, 3309911, 3027104, 2385466, 3173593, 3270988, 2861136, 2684068, 2927024, 3570789, 3458591, 2720017, 2798556, 3209667, 2116427, 2717878, 3115320, 3063212, 3391538, 2625443, 1866179, 3556122, 3160130, 2649363, 2598032, 2460730, 3252843, 2353318, 2824108, 4076632, 3015190, 2881253, 2884679, 2866046, 4062943, 3336998, 2296914, 3093374, 4602653, 2877288, 2570633, 3238787, 3380645, 2758141, 2477442, 3188338, 3443529, 2511792, 2155552, 2635721, 3734293

In [26]:
f_star = [(-1)**i * f[i] for i in range(len(f))]
print(f)
print(f_star)

[0, 0, 4, 1, 4, 1, 0, 1, 3, 2, 0, 3, 6, 0, 3, 2, 3, 0, 0, 1, 0, 4, 0, 2, 0, 2, 3, 3, 5, 2, 0, 1, 2, 2, 0, 1, 2, 0, 1, 4, 0, 1, 1, 3, 0, 1, 1, 2, 4, 1, 2, 0, 4, 0, 1, 6, 0, 4, 1, 0, 1, 2, 3, 0, 2, 0, 3, 2, 2, 1, 2, 2, 3, 3, 3, 0, 0, 3, 2, 1, 1, 0, 0, 4, 4, 0, 4, 0, 0, 1, 2, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 5, 4, 4, 2, 7, 0, 1, 1, 4, 0, 2, 3, 1, 3, 2, 2, 0, 1, 5, 0, 0, 2, 3, 2, 2, 2, 3, 2, 1, 1, 0, 1, 0, 1, 3, 1, 4, 0, 4, 3, 3, 0, 1, 3, 1, 3, 5, 3, 0, 1, 1, 2, 2, 1, 0, 4, 5, 4, 3, 5, 1, 1, 2, 1, 1, 2, 3, 0, 4, 0, 0, 4, 1, 2, 0, 1, 3, 0, 4, 3, 0, 2, 0, 0, 3, 1, 6, 0, 5, 7, 0, 2, 2, 1, 0, 2, 0, 5, 1, 4, 0, 0, 1, 1, 2, 1, 3, 4, 5, 1, 0, 0, 1, 4, 2, 0, 0, 2, 4, 3, 3, 3, 5, 1, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 1, 0, 2, 3, 1, 0, 0, 4, 1, 3, 0, 1, 0, 1, 4, 0, 1, 2, 1, 2, 3, 0, 3, 5, 4, 4, 1, 2, 1, 1, 1, 1, 5, 5, 1, 2, 5, 1, 4, 1, 0, 0, 2, 6, 0, 4, 4, 0, 0, 0, 7, 4, 4, 1, 0, 0, 0, 0, 0, 1, 4, 2, 4, 1, 6, 2, 9, 2, 2, 3, 2, 1, 0, 0, 2, 2, 0, 1, 4, 3, 3, 3, 1, 1, 0, 2, 0, 0, 0, 4, 1, 7, 0, 0, 1, 2, 3, 4, 

In [27]:
f = NegaCyclic_Conv(f,f_star,0)
f = f[::2]
print(f)
f_star = [(-1)**i * f[i] for i in range(len(f))]
print(f_star)
f = NegaCyclic_Conv(f,f_star,0)
print(f)

[-132, -179, 34, -133, -217, -161, -201, -155, -171, -160, -128, -98, -55, -53, -139, -123, -82, -53, -127, -177, -182, -136, -132, -134, -182, -175, -101, -149, -187, -200, -372, -99, 7, -41, 57, -1, 68, -2, -60, -5, -97, -156, -176, 182, -139, -111, -262, 27, -70, -382, -382, -112, -182, -240, -142, -287, -510, -227, -232, -377, -223, -106, -341, -262, -656, -268, -15, -24, -98, -140, -111, -308, -582, -393, -243, -147, -146, -165, -256, -247, -299, -141, -159, -184, -37, -283, -332, -199, -308, -390, -358, -168, -189, -325, -187, -257, -92, -356, -320, -64, -166, -138, -138, -256, -348, -246, -138, -336, -183, -183, -163, 27, -396, -69, -109, -106, -316, -57, -329, -161, -332, -349, -318, -78, -435, -160, -164, -355, -186, -269, -437, 6, -322, -247, -199, -63, -283, -117, -96, -463, -333, -287, -208, -161, -266, -219, -440, -97, -352, -268, -258, -242, -381, -397, -119, -362, -346, -293, -325, -255, -275, -513, -537, -502, -520, -123, 46, -432, -482, 88, -324, -292, -178, -213, -167

In [28]:
h = [-17719, 836349, 1515114, 1467820, 720699, 794357, 2449564, 1240444, 779544, 1056357, 1370347, 1929573, 1314203, 1573444, 2435669, 1969618, 994861, 1906460, 2108782, 2996757, 1416004, 1090614, 1696011, 2406608, 1628680, 2202682, 2389811, 2764218, 2266100, 1828644, 2534785, 2911275, 1733201, 2092121, 2585275, 2139269, 3338229, 1884820, 2052590, 1751170, 2376562, 2057412, 1536541, 2383907, 2352876, 1669564, 1740019, 2587122, 2750630, 2217788, 1986948, 2893778, 2562617, 2780016, 1666804, 2264183, 3339919, 2742232, 3052997, 3029941, 3309911, 3027104, 2385466, 3173593, 3270988, 2861136, 2684068, 2927024, 3570789, 3458591, 2720017, 2798556, 3209667, 2116427, 2717878, 3115320, 3063212, 3391538, 2625443, 1866179, 3556122, 3160130, 2649363, 2598032, 2460730, 3252843, 2353318, 2824108, 4076632, 3015190, 2881253, 2884679, 2866046, 4062943, 3336998, 2296914, 3093374, 4602653, 2877288, 2570633, 3238787, 3380645, 2758141, 2477442, 3188338, 3443529, 2511792, 2155552, 2635721, 3734293, 3309044, 2233831, 3286863, 2989702, 2714723, 2160829, 3409047, 3392354, 2474599, 3085299, 3209789, 3608029, 2671357, 2005876, 2043761, 2985364, 1337391, 2546892, 1935780, 3069797, 2288379, 2148967, 2424805, 2506329, 2762636, 2684835, 2392092, 2973440, 2764708, 2264403, 2233302, 3151992, 2741510, 3124294, 2513083, 3022090, 2893781, 2444895, 2279554, 2905770, 2850937, 2884069, 1805614, 2246756, 3112220, 2234405, 1985640, 2048931, 1804958, 2443776, 1727477, 2225309, 2109374, 3033923, 940356, 2507534, 2871501, 2130000, 1815224, 1922307, 2061637, 2193398, 2182954, 1568596, 2327156, 1918351, 1269169, 585030, 2185748, 2640746, 1891577, 1923815, 2021646, 2445980, 1227923, 2112225, 1466955, 1795572, 1833918, 1029111, 1219834, 1537292, 1345645, 452582, 1403651, 1444080, 1235327, 307227, 1332081, 746660, 1184612, -219477, 199030, 1080963, 674275, 13074, -146798, 32992, 102430, -503846, -398730, 310659, 101756, -370969, 48726, 484146, 718063, 39901, -670114, 1164766, 67477, -1378281, -516671, -305768, 341209, -402040, 169006, 189187, 193073, -745304, -279230, -649750, 884175, -1039518, -795839, -978375, -800126, -1356988, -2206944, -1238863, -1519336, -1580224, -1562125, -1241262, -1366808, -658558, -1387790, -1357414, -536251, -512231, -881104, -1837127, -568034, -1455885, -766112, -2134820] 
print(check_arrays(temp[ptrNf:ptrNf+hn],h,P[0]))


exactly equal
True


In [30]:
inv2s = [16775169,16770049,16769025,16766977]

inv4s = [mod_mul_standard(inv2s[i], inv2s[i], P[i]) for i in range(4)]
print(inv4s)
mu_inv4s = [Mu(inv4s[i], P[i]) for i in range(4)]
print(mu_inv4s)

[-8387584, -8385024, -8384512, -8383488]
[-536870894, -536870894, -536870894, -536870894]
